In [ ]:
# Installing required libraries
# pip install dataframe_image

In [ ]:
# Importing the librarieimport boto3 
import botocore 
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
from sklearn.preprocessing import minmax_scale
import matplotlib.pyplot as plt
import seaborn as sns 
import dataframe_image as dfi
import boto3
import io
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Showing 30 rows and unlimited columns
pd.set_option('display.max_rows', 30)
pd.options.display.max_columns = None

In [ ]:
# Specifying the bucket
bucket='daen690-meraki-data'
dashboard_bucket = 'daen690-meraki-dashboard'

In [ ]:
# Specifying the data file name and location for receiving dataset
data_receiving = 'gmu_training_receiving_2.xlsx'
data_path_receiving = 's3://{}/{}'.format(bucket, data_receiving)

In [ ]:
# Specifying the data file name and location for locations dataset
data_locations = 'gmu_training_locations.xlsx'
data_path_locations = 's3://{}/{}'.format(bucket, data_locations)

In [ ]:
# Specifying the data file name and location for disposals dataset
data_disposals = 'gmu_training_disposals.xlsx'
data_path_disposals = 's3://{}/{}'.format(bucket, data_disposals)

---

# Receiving Dataset

In [ ]:
'''import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
bucket= 'daen690-meraki-data'
data_key = 'gmu_training_receiving_2.xlsx'
data_location = 's3://{}/{}'.format(bucket, data_key)

df_receiving = pd.read_excel(data_location)
df_receiving'''

In [ ]:
df_receiving = pd.read_excel(data_path_receiving)
df_receiving

In [ ]:
# Checking data dimensions
df_receiving.shape

# Dataset has 53242 rows × 13 columns

In [ ]:
# Checking columns, data type and missing values
df_receiving.info()

In [ ]:
df_receiving.isna().sum()

# There are 49,757 missing values in the column 'MISCLASSIFIED_FAP', rest all columns don't have any missing values 

In [ ]:
# Checking for duplicated rows
df_receiving[df_receiving.duplicated()]

# there are no duplicated rows in this dataset

In [ ]:
# Checking the column names
df_receiving.columns

In [ ]:
# Renaming some of the column names
df_receiving.columns = ['TRANS_NUMBER', 'BUSINESS_UNIT', 'CREATION_METHOD', 'OPR_ID', 
       'ASSET_SUBTYPE', 'ASSET_CLASS', 'ASSET_CLASS_DESCR',
       'ACQUISITION_DT', 'AGENCY_CODE', 'COST', 'LOCATION_TYPE',
       'FAP_LIKE_FLAG', 'MISCLASSIFIED_FAP']

df_receiving.head()

In [ ]:
# Changing the data type of 'TRANS_NUMBER' and 'ASSET_CLASS' to string 
df_receiving['TRANS_NUMBER'] = df_receiving['TRANS_NUMBER'].astype('str')
df_receiving['ASSET_CLASS'] = df_receiving['ASSET_CLASS'].astype('str')

In [ ]:
# Checking for data types of all columns again
df_receiving.info()

In [ ]:
# Checking details for numerical columns
df_receiving.describe()

In [ ]:
# Histogram of Asset Cost
sns.distplot(df_receiving.COST, kde=True).set_title('Histogram of Asset Cost')

In [ ]:
# Checking details for categorical columns
df_receiving.describe(include=np.object)

In [ ]:
# Checking for unique values and count for "CREATION_METHOD" column
df_receiving.groupby("CREATION_METHOD").agg(
    TRANSACTION_COUNT = ("CREATION_METHOD", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()

In [ ]:
df_receiving.CREATION_METHOD = df_receiving.CREATION_METHOD.replace(
    ['AM Page', 'PI Add'], ['Manual Creation', 'Manual Creation'])

In [ ]:
# Checking for unique values and count for "CREATION_METHOD" column again
df_receiving.groupby("CREATION_METHOD").agg(
    TRANSACTION_COUNT = ("CREATION_METHOD", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()

In [ ]:
# Creation Method-wise Transaction Count
sns.countplot(x="CREATION_METHOD", data=df_receiving).set_title('Creation Method-wise Transaction Count')

In [ ]:
# Checking for unique values and count for "ASSET_SUBTYPE" column
ast_typ_count = df_receiving.groupby("ASSET_SUBTYPE").agg(
                    TRANSACTION_COUNT = ("ASSET_SUBTYPE", "count"),
                ).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()
ast_typ_count

In [ ]:
# Asset Subtype-wise Transaction Count
p = sns.barplot(x='ASSET_SUBTYPE', y='TRANSACTION_COUNT', data=ast_typ_count)
p.set_xticklabels(p.get_xticklabels(),rotation=90)
p.set_title('Asset Subtype-wise Transaction Count')

In [ ]:
# Checking for unique values and count for "LOCATION_TYPE" column
loc_typ_count = df_receiving.groupby("LOCATION_TYPE").agg(
    TRANSACTION_COUNT = ("LOCATION_TYPE", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()
loc_typ_count

In [ ]:
# Location type-wise Transaction Count
p = sns.barplot(x='LOCATION_TYPE', y='TRANSACTION_COUNT', data=loc_typ_count)
p.set_xticklabels(p.get_xticklabels(),rotation=90)
p.set_title('Location type-wise Transaction Count')

In [ ]:
# Checking for unique values and count for "FAP_LIKE_FLAG" column
fap_flag_count = df_receiving.groupby("FAP_LIKE_FLAG").agg(
                    TRANSACTION_COUNT = ("FAP_LIKE_FLAG", "count"),
                ).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()
fap_flag_count

In [ ]:
# FAP Like Flag-wise Transaction Count
p = sns.barplot(x='FAP_LIKE_FLAG', y='TRANSACTION_COUNT', data=fap_flag_count)
p.set_title('FAP Like Flag-wise Transaction Count')

In [ ]:
# Checking for unique values and count for "MISCLASSIFIED_FAP" column
df_receiving.groupby("MISCLASSIFIED_FAP").agg(
    TRANSACTION_COUNT = ("MISCLASSIFIED_FAP", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()

In [ ]:
df_receiving.MISCLASSIFIED_FAP = df_receiving.MISCLASSIFIED_FAP.fillna('Correct_classification')

In [ ]:
# Checking for unique values and count for "MISCLASSIFIED_FAP" column again
miscl_fap_count = df_receiving.groupby("MISCLASSIFIED_FAP").agg(
    TRANSACTION_COUNT = ("MISCLASSIFIED_FAP", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()
miscl_fap_count

In [ ]:
# Misclassified FAP-wise Transaction Count
p = sns.barplot(x='MISCLASSIFIED_FAP', y='TRANSACTION_COUNT', data=miscl_fap_count)
p.set_xticklabels(p.get_xticklabels(),rotation=70)
p.set_title('Misclassified FAP-wise Transaction Count')

In [ ]:
# Creating a column 'CORRECT_CLASSIFICATION'
mis_list = ['Should be FAP based on Asset Class', 'Should not be FAP based on Asset Class', 'FAP not in Approved Location']
list_cor_class = []

for idx in df_receiving.index:
    cor = df_receiving['MISCLASSIFIED_FAP'][idx]
    if cor in mis_list:
        list_cor_class.append("No")
    elif cor == 'Correct_classification':
        list_cor_class.append("Yes")

In [ ]:
print(len(list_cor_class))

In [ ]:
df_receiving.insert(13, 'CORRECT_CLASSIFICATION', list_cor_class)

In [ ]:
df_receiving.head()

In [ ]:
# To check values in MISCLASSIFIED_FAP for CORRECT_CLASSIFICATION = No 
df_receiving[df_receiving['CORRECT_CLASSIFICATION'] == 'No']

In [ ]:
df_receiving.info()

In [ ]:
# Looking at top users based on number of transactions
df_receiving.groupby("OPR_ID").agg(
    TRANSACTION_COUNT = ("COST", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index().round(2)

In [ ]:
# Showing 100 rows 
pd.set_option('display.max_rows', 100)

In [ ]:
# Filtering incorrect classifications and finding top users based on number of transactions
user_trans = df_receiving[df_receiving['CORRECT_CLASSIFICATION'].str.contains('No')].groupby("OPR_ID").agg(
            TRANSACTION_COUNT = ("COST", "count"),
            ).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index().round(2)
user_trans

In [ ]:
# Plot for top users with maximum erroneous transactions
sns.barplot(x="OPR_ID", y="TRANSACTION_COUNT", data=user_trans)
plt.title("Top users with maximum erroneous transactions")
plt.xticks(rotation=90)
plt.gcf().set_size_inches( 16, 10)

In [ ]:
# Looking at top users based on total asset cost that they are handling
df_receiving.groupby("OPR_ID").agg(
    TOTAL_ASSET_COST = ("COST", "sum"),
).sort_values(by = "TOTAL_ASSET_COST", ascending= False).reset_index().round(2)

In [ ]:
#  Filtering incorrect classifications and finding top users based on total asset cost that they are handling
user_tot_cost = df_receiving[df_receiving['CORRECT_CLASSIFICATION'].str.contains('No')].groupby("OPR_ID").agg(
                TOTAL_ASSET_COST = ("COST", "sum"),
                ).sort_values(by = "TOTAL_ASSET_COST", ascending= False).reset_index().round(2)
user_tot_cost

In [ ]:
# Plot for top erring users based on total assets cost
sns.barplot(x="OPR_ID", y="TOTAL_ASSET_COST", data=user_tot_cost)
plt.title("Top erring users based on total assets cost")
plt.xticks(rotation=90)
plt.gcf().set_size_inches(16, 10)

In [ ]:
# Identifying top users based on total asset cost, count and average cost 
df_receiving.groupby("OPR_ID").agg(
    TOTAL_ASSET_COST = ("COST", "sum"),
    TRANSACTION_COUNT = ("COST", "count"),
    AVG_ASSET_COST = ("COST", "mean")
).sort_values(by = "AVG_ASSET_COST", ascending= False).reset_index().round(2)

In [ ]:
# Filtering incorrect classifications and identifying top users based on total asset cost, count and average cost 
user_avg_cost = df_receiving[df_receiving['CORRECT_CLASSIFICATION'].str.contains('No')].groupby("OPR_ID").agg(
                TOTAL_ASSET_COST = ("COST", "sum"),
                TRANSACTION_COUNT = ("COST", "count"),
                AVG_ASSET_COST = ("COST", "mean")
                ).sort_values(by = "AVG_ASSET_COST", ascending= False).reset_index().round(2)
user_avg_cost

In [ ]:
# Plot for top erring users based on average assets cost
sns.barplot(x="OPR_ID", y="AVG_ASSET_COST", data=user_avg_cost)
plt.title("Top erring users based on average asset cost")
plt.xticks(rotation=90)
plt.gcf().set_size_inches(16, 10)

In [ ]:
# Filtering incorrect classifications and identifying top BUSINESS_UNIT based on total asset cost, count and average cost 
bu_rec_totcost = df_receiving[df_receiving['CORRECT_CLASSIFICATION'].str.contains('No')].groupby("BUSINESS_UNIT").agg(
                TOTAL_ASSET_COST = ("COST", "sum"),
                TRANSACTION_COUNT = ("COST", "count"),
                AVG_ASSET_COST = ("COST", "mean")
                ).sort_values(by = "TOTAL_ASSET_COST", ascending= False).reset_index().round(2)
bu_rec_totcost

In [ ]:
# Filtering incorrect classifications and identifying top AGENCY_CODE based on total asset cost, count and average cost 
agc_rec_totcost = df_receiving[df_receiving['CORRECT_CLASSIFICATION'].str.contains('No')].groupby("AGENCY_CODE").agg(
                TOTAL_ASSET_COST = ("COST", "sum"),
                TRANSACTION_COUNT = ("COST", "count"),
                AVG_ASSET_COST = ("COST", "mean")
                ).sort_values(by = "TOTAL_ASSET_COST", ascending= False).reset_index().round(2)
agc_rec_totcost

In [ ]:
# Identifying Business_Unit-wise top users based on total asset cost
df_receiving.groupby(["BUSINESS_UNIT","OPR_ID"]).agg(
    TRANSACTION_COUNT = ("COST", "count"),
    TOTAL_ASSET_COST = ("COST", "sum")
).sort_values(by = ["BUSINESS_UNIT", "TOTAL_ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
# Filtering incorrect classifications and identifying Business_Unit-wise top users based on total asset cost
bu_user = df_receiving[df_receiving['CORRECT_CLASSIFICATION'].str.contains('No')].groupby(["BUSINESS_UNIT","OPR_ID"]).agg(
            TRANSACTION_COUNT = ("COST", "count"),
            TOTAL_ASSET_COST = ("COST", "sum")
            ).sort_values(by = ["BUSINESS_UNIT", "TOTAL_ASSET_COST"], ascending=[True, False]).reset_index().round(2)
bu_user

In [ ]:
# Identifying AGENCY_CODE-wise top users based on total asset cost
df_receiving.groupby(["AGENCY_CODE","OPR_ID"]).agg(
    TRANSACTION_COUNT = ("COST", "count"),
    TOTAL_ASSET_COST = ("COST", "sum")
).sort_values(by = ["AGENCY_CODE", "TOTAL_ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
# Filtering incorrect classifications and identifying AGENCY_CODE-wise top users based on total asset cost
df_receiving[df_receiving['CORRECT_CLASSIFICATION'].str.contains('No')].groupby(["AGENCY_CODE","OPR_ID"]).agg(
    TRANSACTION_COUNT = ("COST", "count"),
    TOTAL_ASSET_COST = ("COST", "sum")
).sort_values(by = ["AGENCY_CODE", "TOTAL_ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
# Creating a new dataframe rec_df by subsetting key attributes influencing error rate
rec_df = df_receiving[['BUSINESS_UNIT','CREATION_METHOD', 'OPR_ID', 'AGENCY_CODE', 'COST', 'CORRECT_CLASSIFICATION']]

In [ ]:
rec_df.head()

In [ ]:
rec_df.describe(include = np.object)

In [ ]:
rec_df.CREATION_METHOD.value_counts()

In [ ]:
# Changing the levels within CREATION_METHOD to '0' and '1'
rec_df.CREATION_METHOD = rec_df.CREATION_METHOD.replace(['Purch. Req', 'Manual Creation'], ['0', '1'])

In [ ]:
rec_df.CREATION_METHOD.value_counts()

In [ ]:
rec_df.CORRECT_CLASSIFICATION.value_counts()

In [ ]:
# Changing the levels within CORRECT_CLASSIFICATION to '0' and '1'
rec_df.CORRECT_CLASSIFICATION = rec_df.CORRECT_CLASSIFICATION.replace(['Yes', 'No'], ['0', '1'])

In [ ]:
rec_df.CORRECT_CLASSIFICATION.value_counts()

In [ ]:
rec_df.head()

In [ ]:
# Renaming the columns for differentiation
rec_df.columns = ['REC_LOCATION', 'REC_CREATION', 'REC_USER', 'REC_AGENCY', 'REC_COST', 'REC_CLASS']

In [ ]:
rec_df.head()

In [ ]:
rec_df.groupby(['REC_USER','REC_LOCATION', 'REC_AGENCY', 'REC_CREATION','REC_CLASS']).agg(
    TRANS_COUNT = ("REC_COST", "count"),
    ASSET_COST = ("REC_COST", "sum")
).sort_values(by = ["REC_USER", "ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
rec_df[(rec_df.REC_CREATION =='1') | (rec_df.REC_CLASS =='1')].groupby(['REC_USER','REC_LOCATION', 'REC_AGENCY', 'REC_CREATION','REC_CLASS']).agg(
    TRANS_COUNT = ("REC_COST", "count"),
    ASSET_COST = ("REC_COST", "sum")
).sort_values(by = ["REC_USER", "ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
# Creating a dataframe with REC_CREATION =='1' and obtaining user-wise count and total asset cost
rec_creation = rec_df[rec_df.REC_CREATION =='1'].groupby(['REC_USER', 'REC_CREATION']).agg(
                    REC_TRANS_COUNT = ("REC_COST", "count"),
                    REC_ASSET_COST = ("REC_COST", "sum")
                ).sort_values(by = ["REC_USER", "REC_ASSET_COST"], ascending=[True, False]).reset_index().round(2)
rec_creation

In [ ]:
# Creating a dataframe with REC_CLASS =='1' and obtaining user-wise count and total asset cost
rec_class = rec_df[rec_df.REC_CLASS =='1'].groupby(['REC_USER','REC_CLASS']).agg(
                REC_TRANS_COUNT = ("REC_COST", "count"),
                REC_ASSET_COST = ("REC_COST", "sum")
            ).sort_values(by = ["REC_USER", "REC_ASSET_COST"], ascending=[True, False]).reset_index().round(2)
rec_class

In [ ]:
#  Creating a dataframe with REC_CREATION =='1' and obtaining location-wise user count
loc_1 = rec_df[rec_df.REC_CREATION =='1'].groupby(['REC_LOCATION','REC_USER']).agg(
            COUNT = ("REC_USER", "count") 
        ).sort_values(by = ['REC_LOCATION', 'COUNT'], ascending= [True, False] ).reset_index()
loc_1

In [ ]:
#  Creating a dataframe with REC_CLASS =='1' and obtaining location-wise user count
loc_2 = rec_df[rec_df.REC_CLASS =='1'].groupby(['REC_LOCATION','REC_USER']).agg(
            COUNT = ("REC_USER", "count") 
        ).sort_values(by = ['REC_LOCATION', 'COUNT'], ascending= [True, False] ).reset_index()
loc_2

---

# Locations Dataset

In [ ]:
df_locations = pd.read_excel(data_path_locations)
df_locations

In [ ]:
df_locations.shape

In [ ]:
df_locations.info()

In [ ]:
# Checking for duplicated rows
df_locations[df_locations.duplicated()]

In [ ]:
# Checking column names
df_locations.columns

In [ ]:
# Renaming column names
df_locations.columns = ['TRANS_NUMBER', 'BUSINESS_UNIT', 'AGENCY_CODE', 'ASSET_SUBTYPE',
       'LOCATION_DATE', 'DOC_NUM', 'LOCATION_TYPE', 'FORM_CHECK',
       'ENTERED_BY', 'COST', 'LOCATION', 'VALID_DOC_FLAG']

df_locations.head()

In [ ]:
# Changing the data type of 'TRANS_NUMBER' and 'VALID_DOC_FLAG' to string
df_locations.TRANS_NUMBER = df_locations.TRANS_NUMBER.astype('str')
df_locations.VALID_DOC_FLAG = df_locations.VALID_DOC_FLAG.astype('str')

In [ ]:
# Converting 'COST' to data type float
# df_locations.COST = df_locations.COST.astype('float')

In [ ]:
df_locations.VALID_DOC_FLAG = df_locations.VALID_DOC_FLAG.replace(['1', '0'], ['Yes', 'No'])

In [ ]:
# Converting 'COST' to data type float
# df_locations.COST = df_locations.COST.astype('float')

In [ ]:
df_locations[df_locations['COST'].str.contains("'-", na=False)]

In [ ]:
df_locations.loc[53738, 'COST'] = 445.9
df_locations.loc[81087, 'COST'] = 334.45
df_locations.loc[81786, 'COST'] = 114.64
df_locations.loc[[97261,97480], 'COST'] = 135.52

In [ ]:
df_locations.COST = df_locations.COST.astype('float')

In [ ]:
df_locations.info()

In [ ]:
df_locations = df_locations.dropna(subset=['ENTERED_BY'])

In [ ]:
df_locations.info()

In [ ]:
df_locations.describe()

In [ ]:
df_locations.describe(include=np.object)

In [ ]:
# Checking for unique values and count for "ASSET_SUBTYPE" column
df_locations.groupby("ASSET_SUBTYPE").agg(
    TRANSACTION_COUNT = ("ASSET_SUBTYPE", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()

In [ ]:
# Checking for unique values and count for "VALID_DOC_FLAG" column
df_locations.groupby("VALID_DOC_FLAG").agg(
    TRANSACTION_COUNT = ("VALID_DOC_FLAG", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()

In [ ]:
df_locations.head()

In [ ]:
# Looking at top users based on number of transactions
df_locations.groupby("ENTERED_BY").agg(
    TRANSACTION_COUNT = ("COST", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index().round(2)

In [ ]:
# Filtering incorrect classifications and finding top users based on number of transactions
user_loc_trans = df_locations[df_locations['VALID_DOC_FLAG'].str.contains('No')].groupby("ENTERED_BY").agg(
            TRANSACTION_COUNT = ("COST", "count"),
            ).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index().round(2)
user_loc_trans

In [ ]:
# Plot for top 50 users with maximum erroneous transactions
sns.barplot(x="ENTERED_BY", y="TRANSACTION_COUNT", data=user_loc_trans[:50])
plt.title("Top users with maximum erroneous transactions")
plt.xticks(rotation=90)
plt.gcf().set_size_inches(16, 10)

In [ ]:
# Looking at top users based on total asset cost that they are handling
df_locations.groupby("ENTERED_BY").agg(
    TOTAL_ASSET_COST = ("COST", "sum"),
).sort_values(by = "TOTAL_ASSET_COST", ascending= False).reset_index().round(2)

In [ ]:
#  Filtering incorrect classifications and finding top users based on total asset cost that they are handling
user_loc_totcost = df_locations[df_locations['VALID_DOC_FLAG'].str.contains('No')].groupby("ENTERED_BY").agg(
                TOTAL_ASSET_COST = ("COST", "sum"),
                ).sort_values(by = "TOTAL_ASSET_COST", ascending= False).reset_index().round(2)
user_loc_totcost

In [ ]:
# Plot for top erring users based on total assets cost
sns.barplot(x="ENTERED_BY", y="TOTAL_ASSET_COST", data=user_loc_totcost[:50])
plt.title("Top erring users based on total assets cost")
plt.xticks(rotation=90)
plt.gcf().set_size_inches(16, 10)

In [ ]:
# Identifying top users based on total asset cost, count and average cost 
df_locations.groupby("ENTERED_BY").agg(
    TOTAL_ASSET_COST = ("COST", "sum"),
    TRANSACTION_COUNT = ("COST", "count"),
    AVG_ASSET_COST = ("COST", "mean")
).sort_values(by = "AVG_ASSET_COST", ascending= False).reset_index().round(2)

In [ ]:
# Filtering incorrect classifications and identifying top users based on total asset cost, count and average cost 
user_loc_avgcost = df_locations[df_locations['VALID_DOC_FLAG'].str.contains('No')].groupby("ENTERED_BY").agg(
                TOTAL_ASSET_COST = ("COST", "sum"),
                TRANSACTION_COUNT = ("COST", "count"),
                AVG_ASSET_COST = ("COST", "mean")
                ).sort_values(by = "AVG_ASSET_COST", ascending= False).reset_index().round(2)
user_loc_avgcost

In [ ]:
# Plot for top erring users based on average assets cost
sns.barplot(x="ENTERED_BY", y="AVG_ASSET_COST", data=user_loc_avgcost[:50])
plt.title("Top erring users based on average asset cost")
plt.xticks(rotation=90)
plt.gcf().set_size_inches(16, 10)

In [ ]:
df_locations.head(3)

In [ ]:
# Identifying Business_Unit-wise top users based on total asset cost
df_locations.groupby(["BUSINESS_UNIT","ENTERED_BY"]).agg(
    TRANSACTION_COUNT = ("COST", "count"),
    TOTAL_ASSET_COST = ("COST", "sum")
).sort_values(by = ["BUSINESS_UNIT", "TOTAL_ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
# Filtering incorrect classifications and identifying Business_Unit-wise top users based on total asset cost
bu_loc_user = df_locations[df_locations['VALID_DOC_FLAG'].str.contains('No')].groupby(["BUSINESS_UNIT","ENTERED_BY"]).agg(
            TRANSACTION_COUNT = ("COST", "count"),
            TOTAL_ASSET_COST = ("COST", "sum")
            ).sort_values(by = ["BUSINESS_UNIT", "TOTAL_ASSET_COST"], ascending=[True, False]).reset_index().round(2)
bu_loc_user

In [ ]:
# Identifying AGENCY_CODE-wise top users based on total asset cost
df_locations.groupby(["AGENCY_CODE","ENTERED_BY"]).agg(
    TRANSACTION_COUNT = ("COST", "count"),
    TOTAL_ASSET_COST = ("COST", "sum")
).sort_values(by = ["AGENCY_CODE", "TOTAL_ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
# Filtering incorrect classifications and identifying AGENCY_CODE-wise top users based on total asset cost
df_locations[df_locations['VALID_DOC_FLAG'].str.contains('No')].groupby(["AGENCY_CODE","ENTERED_BY"]).agg(
    TRANSACTION_COUNT = ("COST", "count"),
    TOTAL_ASSET_COST = ("COST", "sum")
).sort_values(by = ["AGENCY_CODE", "TOTAL_ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
df_locations.head()

In [ ]:
# Creating a new dataframe loc_df by subsetting key attributes influencing error rate
loc_df = df_locations[['BUSINESS_UNIT','AGENCY_CODE', 'ENTERED_BY', 'COST', 'VALID_DOC_FLAG']]

In [ ]:
loc_df.VALID_DOC_FLAG.value_counts()

In [ ]:
# Changing the levels for VALID_DOC_FLAG to '0' and '1'
loc_df.VALID_DOC_FLAG = loc_df.VALID_DOC_FLAG.replace(['Yes', 'No'], ['0', '1'])

In [ ]:
loc_df.VALID_DOC_FLAG.value_counts()

In [ ]:
loc_df.head()

In [ ]:
# Renaming the columns for differentiation
loc_df.columns = ['LOC_LOCATION', 'LOC_AGENCY', 'LOC_USER', 'LOC_COST', 'LOC_DOC']

In [ ]:
loc_df.head()

In [ ]:
loc_df.groupby(['LOC_USER','LOC_LOCATION', 'LOC_AGENCY', 'LOC_DOC']).agg(
    LOC_TRANS_COUNT = ("LOC_COST", "count"),
    LOC_ASSET_COST = ("LOC_COST", "sum")
).sort_values(by = ["LOC_USER", "LOC_ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
# Creating a dataframe with LOC_DOC =='1' and user-wise transaction count and total asset cost 
loc_doc = loc_df[loc_df.LOC_DOC =='1'].groupby(
                ['LOC_USER', 'LOC_DOC']).agg(
                LOC_TRANS_COUNT = ("LOC_COST", "count"),
                LOC_ASSET_COST = ("LOC_COST", "sum")
            ).sort_values(by = ["LOC_USER", "LOC_ASSET_COST"], ascending=[True, False]).reset_index().round(2)
loc_doc

In [ ]:
# Creating a dataframe for LOC_DOC =='1' and location-wise user list
loc_3 = loc_df[loc_df.LOC_DOC =='1'].groupby(['LOC_LOCATION','LOC_USER']).agg(
            COUNT = ("LOC_USER", "count") 
        ).sort_values(by = ['LOC_LOCATION', 'COUNT'], ascending= [True, False]).reset_index()
loc_3

---

# Disposals Dataset

In [ ]:
df_disposals = pd.read_excel(data_path_disposals)
df_disposals

In [ ]:
df_disposals.shape

In [ ]:
df_disposals.info()

In [ ]:
# Checking for duplicated rows
df_disposals[df_disposals.duplicated()]

In [ ]:
# Checking the column names
df_disposals.columns

In [ ]:
# Changing the column names
df_disposals.columns = ['TRANS_NUMBER', 'BUSINESS_UNIT', 'ASSET_DESCR', 'ASSET_SUBTYPE', 'DISP_DOC',
       'EXPECTED_RET_DATE', 'ACTUAL_RET_DATE', 'SCAN_TYPE', 'COST', 'USER_ID']
df_disposals.head()

In [ ]:
# Changing the data type of 'TRANS_NUMBER' to string and 'COST' to 'float' 
df_disposals['TRANS_NUMBER'] = df_disposals['TRANS_NUMBER'].astype('str')

In [ ]:
# df_disposals['COST'] = df_disposals['COST'].astype('float')

In [ ]:
df_disposals[df_disposals['COST'].str.contains("'-", na=False)]

In [ ]:
df_disposals.loc[25853, 'COST'] = 61.58

In [ ]:
df_disposals['COST'] = df_disposals['COST'].astype('float')

In [ ]:
# Checking for data types of all columns again
df_disposals.dtypes

In [ ]:
# Checking details for numerical columns
df_disposals.describe()

In [ ]:
# Checking details for categorical columns
df_disposals.describe(include=np.object)

In [ ]:
# Checking for unique values and count for "ASSET_SUBTYPE" column
df_disposals.groupby("ASSET_SUBTYPE").agg(
    TRANSACTION_COUNT = ("ASSET_SUBTYPE", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()

In [ ]:
# Checking for unique values and count for "DISP_DOC" column
df_disposals.groupby("DISP_DOC").agg(
    TRANSACTION_COUNT = ("DISP_DOC", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()

In [ ]:
# Replacing the 'na' values in "SCAN_TYPE" column as 'Missing'
df_disposals.SCAN_TYPE = df_disposals.SCAN_TYPE.fillna('Missing')

In [ ]:
# Checking for unique values and count for "SCAN_TYPE" column
df_disposals.groupby("SCAN_TYPE").agg(
    TRANSACTION_COUNT = ("SCAN_TYPE", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()

In [ ]:
df_disposals.SCAN_TYPE = df_disposals.SCAN_TYPE.replace(
    ['Scan Transfer', 'Discovered', 'Invalid'], ['Scanned', 'Scanned', 'Manual'])

In [ ]:
# Checking for unique values and count for "SCAN_TYPE" column again
df_disposals.groupby("SCAN_TYPE").agg(
    TRANSACTION_COUNT = ("SCAN_TYPE", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()

In [ ]:
# Create a new column 'EXPECTED_RET_YEAR'
list_exp_retyear = []
for idx in df_disposals.index:
    list_exp_retyear.append(df_disposals['EXPECTED_RET_DATE'][idx].split(".")[2]) 

In [ ]:
print(len(list_exp_retyear))

In [ ]:
df_disposals.insert(6, 'EXPECTED_RET_YEAR', list_exp_retyear)

In [ ]:
# Create a new column 'ACTUAL_RET_YEAR'
list_act_retyear = []
for idx in df_disposals.index:
    list_act_retyear.append(df_disposals['ACTUAL_RET_DATE'][idx].split(".")[2]) 

In [ ]:
print(len(list_act_retyear))

In [ ]:
df_disposals.insert(8, 'ACTUAL_RET_YEAR', list_act_retyear)

In [ ]:
df_disposals.head(3)

In [ ]:
# Converting the EXPECTED_RET_YEAR and ACTUAL_RET_YEAR to check the difference between them
df_disposals['EXPECTED_RET_YEAR'] = df_disposals['EXPECTED_RET_YEAR'].astype('int')
df_disposals['ACTUAL_RET_YEAR'] = df_disposals['ACTUAL_RET_YEAR'].astype('int')

In [ ]:
df_disposals.dtypes

In [ ]:
# Create a new column 'CORRECT_RET_YEAR'
list_ret_year = []
for idx in df_disposals.index:
    if(df_disposals['ACTUAL_RET_YEAR'][idx] - df_disposals['EXPECTED_RET_YEAR'][idx]) >=0:
        list_ret_year.append('Yes')
    else:
        list_ret_year.append('No')

In [ ]:
print(len(list_ret_year))

In [ ]:
df_disposals.insert(9, 'CORRECT_RET_YEAR', list_ret_year)

In [ ]:
df_disposals.head(3)

In [ ]:
df_disposals.info()

In [ ]:
# Checking for unique values and count for "CORRECT_RET_YEAR" column again
df_disposals.groupby("CORRECT_RET_YEAR").agg(
    TRANSACTION_COUNT = ("CORRECT_RET_YEAR", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index()

In [ ]:
# Looking at top users based on number of transactions
df_disposals.groupby("USER_ID").agg(
    TRANSACTION_COUNT = ("COST", "count"),
).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index().round(2)

In [ ]:
# Filtering for DISP_DOC = No and finding top users based on number of transactions
user_disp_trans1 = df_disposals[df_disposals['DISP_DOC'].str.contains('No')].groupby("USER_ID").agg(
            TRANSACTION_COUNT = ("COST", "count"),
            ).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index().round(2)
user_disp_trans1

In [ ]:
# Plot for top users with maximum erroneous transactions
sns.barplot(x="USER_ID", y="TRANSACTION_COUNT", data=user_disp_trans1)
plt.title("Top users with maximum transactions with error in Scan Type")
plt.xticks(rotation=90)
plt.gcf().set_size_inches(16, 10)

In [ ]:
# Filtering for SCAN_TYPE = Missing, Manual and finding top users based on number of transactions
user_disp_trans2 = df_disposals[df_disposals.SCAN_TYPE.isin(['Missing', 'Manual'])].groupby("USER_ID").agg(
            TRANSACTION_COUNT = ("COST", "count"),
            ).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index().round(2)
user_disp_trans2

In [ ]:
# Plot for top users with maximum erroneous transactions
sns.barplot(x="USER_ID", y="TRANSACTION_COUNT", data=user_disp_trans2[:50])
plt.title("Top users with maximum transactions with error in Scan Type")
plt.xticks(rotation=90)
plt.gcf().set_size_inches(16, 10)

In [ ]:
# Filtering for CORRECT_RET_YEAR = No and finding top users based on number of transactions
user_disp_trans3 = df_disposals[df_disposals['CORRECT_RET_YEAR'].str.contains('No')].groupby("USER_ID").agg(
            TRANSACTION_COUNT = ("COST", "count"),
            ).sort_values(by = "TRANSACTION_COUNT", ascending= False).reset_index().round(2)
user_disp_trans3

In [ ]:
# Plot for top users with maximum erroneous transactions
sns.barplot(x="USER_ID", y="TRANSACTION_COUNT", data=user_disp_trans3[:50])
plt.title("Top users with maximum transactions with error in Retirement Years")
plt.xticks(rotation=90)
plt.gcf().set_size_inches(16, 10)

In [ ]:
# Looking at top users based on total asset cost that they are handling
df_disposals.groupby("USER_ID").agg(
    TOTAL_ASSET_COST = ("COST", "sum"),
).sort_values(by = "TOTAL_ASSET_COST", ascending= False).reset_index().round(2)

In [ ]:
#  Filtering DISP_DOC = No and finding top users based on total asset cost that they are handling
user_disp_totcost1 = df_disposals[df_disposals['DISP_DOC'].str.contains('No')].groupby("USER_ID").agg(
                TOTAL_ASSET_COST = ("COST", "sum"),
                ).sort_values(by = "TOTAL_ASSET_COST", ascending= False).reset_index().round(2)
user_disp_totcost1

In [ ]:
# Plot for top erring users based on total assets cost
sns.barplot(x="USER_ID", y="TOTAL_ASSET_COST", data=user_disp_totcost1)
plt.title("Top erring users based on total assets cost")
plt.xticks(rotation=90)
plt.gcf().set_size_inches(16, 10)

In [ ]:
# Identifying top users based on total asset cost, count and average cost 
df_disposals.groupby("USER_ID").agg(
    TOTAL_ASSET_COST = ("COST", "sum"),
    TRANSACTION_COUNT = ("COST", "count"),
    AVG_ASSET_COST = ("COST", "mean")
).sort_values(by = "AVG_ASSET_COST", ascending= False).reset_index().round(2)

In [ ]:
# Filtering DISP_DOC = No and identifying top users based on total asset cost, count and average cost 
user_disp_avgcost1 = df_disposals[df_disposals['DISP_DOC'].str.contains('No')].groupby("USER_ID").agg(
                TOTAL_ASSET_COST = ("COST", "sum"),
                TRANSACTION_COUNT = ("COST", "count"),
                AVG_ASSET_COST = ("COST", "mean")
                ).sort_values(by = "AVG_ASSET_COST", ascending= False).reset_index().round(2)
user_disp_avgcost1

In [ ]:
# Plot for top erring users based on average assets cost
sns.barplot(x="USER_ID", y="AVG_ASSET_COST", data=user_disp_avgcost1)
plt.title("Top erring users based on average asset cost")
plt.xticks(rotation=90)
plt.gcf().set_size_inches(16, 10)

In [ ]:
# Identifying Business_Unit-wise top users based on total asset cost
df_disposals.groupby(["BUSINESS_UNIT","USER_ID"]).agg(
    TRANSACTION_COUNT = ("COST", "count"),
    TOTAL_ASSET_COST = ("COST", "sum")
).sort_values(by = ["BUSINESS_UNIT", "TOTAL_ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
# Filtering DISP_DOC = No and identifying Business_Unit-wise top users based on total asset cost
bu_disp_user1 = df_disposals[df_disposals['DISP_DOC'].str.contains('No')].groupby(["BUSINESS_UNIT","USER_ID"]).agg(
            TRANSACTION_COUNT = ("COST", "count"),
            TOTAL_ASSET_COST = ("COST", "sum")
            ).sort_values(by = ["BUSINESS_UNIT", "TOTAL_ASSET_COST"], ascending=[True, False]).reset_index().round(2)
bu_disp_user1

In [ ]:
df_disposals.head()

In [ ]:
# Creating a new dataframe disp_df by subsetting key attributes influencing error rate
disp_df = df_disposals[['BUSINESS_UNIT','DISP_DOC', 'CORRECT_RET_YEAR', 'SCAN_TYPE', 'COST', 'USER_ID']]

In [ ]:
disp_df.head()

In [ ]:
disp_df.DISP_DOC.value_counts()

In [ ]:
# Changing the levels for DISP_DOC to '0' and '1'
disp_df.DISP_DOC = disp_df.DISP_DOC.replace(['Yes', 'No'], ['0', '1'])

In [ ]:
disp_df.DISP_DOC.value_counts()

In [ ]:
disp_df.CORRECT_RET_YEAR.value_counts()

In [ ]:
# Changing the levels for CORRECT_RET_YEAR to '0' and '1'
disp_df.CORRECT_RET_YEAR = disp_df.CORRECT_RET_YEAR.replace(['Yes', 'No'], ['0', '1'])

In [ ]:
disp_df.CORRECT_RET_YEAR.value_counts()

In [ ]:
# Changing the levels for SCAN_TYPE to '0', '0.5' and '1'
disp_df.SCAN_TYPE.value_counts()

In [ ]:
disp_df.SCAN_TYPE = disp_df.SCAN_TYPE.replace(['Scanned', 'Manual', 'Missing'], ['0', '0.5', '1'])

In [ ]:
disp_df.SCAN_TYPE.value_counts()

In [ ]:
disp_df.head()

In [ ]:
disp_df.columns

In [ ]:
# Renaming the columns for differentiation
disp_df.columns = ['DISP_LOCATION', 'DISP_DOC', 'DISP_RET_YEAR', 'DISP_SCAN_TYPE', 'DISP_COST', 'DISP_USER']

In [ ]:
disp_df.head()

In [ ]:
disp_df.groupby(['DISP_USER','DISP_LOCATION', 'DISP_DOC', 'DISP_RET_YEAR','DISP_SCAN_TYPE']).agg(
    DISP_TRANS_COUNT = ("DISP_COST", "count"),
    DISP_ASSET_COST = ("DISP_COST", "sum")
).sort_values(by = ["DISP_USER", "DISP_ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
disp_df[(disp_df.DISP_DOC =='1') | (disp_df.DISP_RET_YEAR =='1') | (disp_df.DISP_SCAN_TYPE =='1') | (disp_df.DISP_SCAN_TYPE =='0.5')].groupby(
    ['DISP_USER','DISP_LOCATION', 'DISP_DOC', 'DISP_RET_YEAR','DISP_SCAN_TYPE']).agg(
    DISP_TRANS_COUNT = ("DISP_COST", "count"),
    DISP_ASSET_COST = ("DISP_COST", "sum")
).sort_values(by = ["DISP_USER", "DISP_ASSET_COST"], ascending=[True, False]).reset_index().round(2)

In [ ]:
# Creating a dataframe for DISP_DOC =='1' and user-wise transaction count and total asset cost
disp_doc = disp_df[disp_df.DISP_DOC =='1'].groupby(
                ['DISP_USER','DISP_DOC']).agg(
                DISP_TRANS_COUNT = ("DISP_COST", "count"),
                DISP_ASSET_COST = ("DISP_COST", "sum")
            ).sort_values(by = ["DISP_USER", "DISP_ASSET_COST"], ascending=[True, False]).reset_index().round(2)
disp_doc

In [ ]:
# Creating a dataframe for DISP_RET_YEAR =='1' and user-wise transaction count and total asset cost
disp_retyear = disp_df[disp_df.DISP_RET_YEAR =='1'].groupby(
                    ['DISP_USER','DISP_RET_YEAR']).agg(
                    DISP_TRANS_COUNT = ("DISP_COST", "count"),
                    DISP_ASSET_COST = ("DISP_COST", "sum")
                ).sort_values(by = ["DISP_USER", "DISP_ASSET_COST"], ascending=[True, False]).reset_index().round(2)
disp_retyear

In [ ]:
# Creating a dataframe for DISP_SCAN_TYPE =='1' and user-wise transaction count and total asset cost
disp_scan_mis = disp_df[(disp_df.DISP_SCAN_TYPE =='1')].groupby(
                ['DISP_USER', 'DISP_SCAN_TYPE']).agg(
                DISP_TRANS_COUNT = ("DISP_COST", "count"),
                DISP_ASSET_COST = ("DISP_COST", "sum")
            ).sort_values(by = ["DISP_USER", "DISP_ASSET_COST"], ascending=[True, False]).reset_index().round(2)
disp_scan_mis

In [ ]:
# Creating a dataframe for DISP_SCAN_TYPE =='0.5' and user-wise transaction count and total asset cost
disp_scan_man = disp_df[(disp_df.DISP_SCAN_TYPE =='0.5')].groupby(
                ['DISP_USER', 'DISP_SCAN_TYPE']).agg(
                DISP_TRANS_COUNT = ("DISP_COST", "count"),
                DISP_ASSET_COST = ("DISP_COST", "sum")
            ).sort_values(by = ["DISP_USER", "DISP_ASSET_COST"], ascending=[True, False]).reset_index().round(2)
disp_scan_man

In [ ]:
# Creating a dataframe for DISP_DOC =='1' and location-wise user list
loc_4 = disp_df[disp_df.DISP_DOC =='1'].groupby(['DISP_LOCATION','DISP_USER']).agg(
            COUNT = ("DISP_USER", "count") 
        ).sort_values(by = ['DISP_LOCATION', 'COUNT'], ascending= [True, False]).reset_index()
loc_4

In [ ]:
# Creating a dataframe for DISP_RET_YEAR =='1' and location-wise user list
loc_5 = disp_df[disp_df.DISP_RET_YEAR =='1'].groupby(['DISP_LOCATION','DISP_USER']).agg(
            COUNT = ("DISP_USER", "count") 
        ).sort_values(by = ['DISP_LOCATION', 'COUNT'], ascending= [True, False]).reset_index()
loc_5

In [ ]:
# Creating a dataframe for DISP_SCAN_TYPE =='0.5' or '1' and location-wise user list
loc_6 = disp_df[(disp_df.DISP_SCAN_TYPE == '0.5') |(disp_df.DISP_SCAN_TYPE == '1')].groupby(['DISP_LOCATION','DISP_USER']).agg(
            COUNT = ("DISP_USER", "count") 
        ).sort_values(by = ['DISP_LOCATION', 'COUNT'], ascending= [True, False]).reset_index()
loc_6

---

# Weighted Scoring Matrix Model

In [ ]:
# Joining dataframes rec_creation, rec_class, loc_doc, disp_doc, disp_retyear, disp_scan_mis, disp_scan_man

In [ ]:
joined_df1 = pd.merge(rec_creation, rec_class, how ='outer', on ='REC_USER') 
joined_df1

In [ ]:
joined_df1.columns

In [ ]:
joined_df1.columns = ['REC_USER', 'REC_CREATION', 'REC_CREAT_COUNT', 'REC_CREAT_COST', 
                      'REC_CLASS', 'REC_CLASS_COUNT', 'REC_CLASS_COST']

In [ ]:
joined_df1.head()

In [ ]:
joined_df2 = pd.merge(disp_doc, disp_retyear, how ='outer', on ='DISP_USER') 
joined_df2

In [ ]:
joined_df2.columns

In [ ]:
joined_df2.columns = ['DISP_USER', 'DISP_DOC', 'DISP_DOC_COUNT', 'DISP_DOC_COST', 
                      'DISP_RET_YEAR', 'DISP_RET_COUNT', 'DISP_RET_COST']

In [ ]:
joined_df2.head()

In [ ]:
joined_df3 = pd.merge(joined_df2, disp_scan_mis, how ='outer', on ='DISP_USER') 
joined_df3

In [ ]:
joined_df3.columns

In [ ]:
joined_df3.columns = ['DISP_USER', 'DISP_DOC', 'DISP_DOC_COUNT', 'DISP_DOC_COST', 
                      'DISP_RET_YEAR', 'DISP_RET_COUNT', 'DISP_RET_COST', 'DISP_SCAN_MIS', 
                      'DISP_SCAN_MIS_COUNT', 'DISP_SCAN_MIS_COST']

In [ ]:
joined_df3.head()

In [ ]:
joined_df4 = pd.merge(joined_df3, disp_scan_man, how ='outer', on ='DISP_USER') 
joined_df4

In [ ]:
joined_df4.columns

In [ ]:
joined_df4.columns = ['DISP_USER', 'DISP_DOC', 'DISP_DOC_COUNT', 'DISP_DOC_COST', 
                      'DISP_RET_YEAR', 'DISP_RET_COUNT', 'DISP_RET_COST', 'DISP_SCAN_MIS', 
                      'DISP_SCAN_MIS_COUNT', 'DISP_SCAN_MIS_COST', 'DISP_SCAN_MAN', 
                      'DISP_SCAN_MAN_COUNT', 'DISP_SCAN_MAN_COST']

In [ ]:
joined_df4.head()

In [ ]:
joined_df5 = pd.merge(joined_df1, loc_doc, how ='outer', left_on ='REC_USER', right_on = 'LOC_USER') 
joined_df5

In [ ]:
joined_df5.columns

In [ ]:
joined_df5.columns = ['REC_USER', 'REC_CREATION', 'REC_CREAT_COUNT', 'REC_CREAT_COST', 
                      'REC_CLASS', 'REC_CLASS_COUNT', 'REC_CLASS_COST', 'LOC_USER', 'LOC_DOC', 
                      'LOC_DOC_COUNT', 'LOC_DOC_COST']

In [ ]:
joined_df5.head()

In [ ]:
joined_df6 = pd.merge(joined_df5, joined_df4, how ='outer', left_on = 'REC_USER', right_on = 'DISP_USER') 
joined_df6

In [ ]:
# Creating single column with USER_ID
list_userid = []

for idx in joined_df6.index:
    for col in joined_df6.columns:
        if col == "REC_USER" and pd.notnull(joined_df6[col][idx]):
            list_userid.append(joined_df6[col][idx])
        elif col == "LOC_USER" and pd.isnull(joined_df6["REC_USER"][idx]) and pd.notnull(joined_df6[col][idx]):
            list_userid.append(joined_df6[col][idx])
        elif col == "DISP_USER" and pd.isnull(joined_df6["REC_USER"][idx]) and pd.isnull(joined_df6["LOC_USER"][idx]) and pd.notnull(joined_df6[col][idx]):
            list_userid.append(joined_df6[col][idx])

len(list_userid)

In [ ]:
joined_df6.insert(0, 'USER_ID', list_userid)
joined_df6.head()

In [ ]:
# Deleting the columns 'REC_USER', 'LOC_USER', 'DISP_USER'
joined_df6 = joined_df6.drop(['REC_USER', 'LOC_USER', 'DISP_USER'], 1)
joined_df6

In [ ]:
joined_df6.info()

In [ ]:
# Creating a list for Receiving Module error flag 
list_module_rec =[]

for idx in joined_df6.index:
    if (joined_df6['REC_CREATION'][idx] == '1') | (joined_df6['REC_CLASS'][idx] == '1'):
        list_module_rec.append(1)
    elif pd.isnull(joined_df6['REC_CREATION'][idx]) and pd.isnull(joined_df6['REC_CLASS'][idx]):
        list_module_rec.append(0)

print(list_module_rec)

In [ ]:
# Creating a list for Location Module error flag 
list_module_loc =[]

for idx in joined_df6.index:
    if joined_df6['LOC_DOC'][idx] == '1':
        list_module_loc.append(1)
    elif pd.isnull(joined_df6['LOC_DOC'][idx]):
        list_module_loc.append(0)
        
print(list_module_loc)

In [ ]:
# Creating a list for Disposal Module error flag 
list_module_disp =[]

for idx in joined_df6.index:
    if (joined_df6['DISP_RET_YEAR'][idx] == '1') | (joined_df6['DISP_SCAN_MIS'][idx] == '1') | (joined_df6['DISP_SCAN_MAN'][idx] == '0.5'):
        list_module_disp.append(1)
    elif pd.isnull(joined_df6['DISP_RET_YEAR'][idx]) and pd.isnull(joined_df6['DISP_SCAN_MIS'][idx]) and pd.isnull(joined_df6['DISP_SCAN_MAN'][idx]):
        list_module_disp.append(0)

print(list_module_disp)

In [ ]:
# Creating a column for error in more than 1 module 
list_module = [x + y + z for x, y, z in zip(list_module_rec, list_module_loc, list_module_disp)]
print(list_module)

In [ ]:
list_more1_module = []

for each in list_module:
    if each > 1:
        list_more1_module.append(1)
    else:
        list_more1_module.append(0)

print(list_more1_module)

In [ ]:
joined_df6.insert(22, 'MORE_1_MODULE', list_more1_module)
joined_df6.head()

In [ ]:
joined_df6.info()

In [ ]:
# Changing the data types of a few columns to numeric
joined_df6['REC_CREATION'] = pd.to_numeric(joined_df6['REC_CREATION'], errors='coerce')
joined_df6['REC_CLASS'] = pd.to_numeric(joined_df6['REC_CLASS'], errors='coerce')
joined_df6['LOC_DOC'] = pd.to_numeric(joined_df6['LOC_DOC'], errors='coerce')
joined_df6['DISP_DOC'] = pd.to_numeric(joined_df6['DISP_DOC'], errors='coerce')
joined_df6['DISP_RET_YEAR'] = pd.to_numeric(joined_df6['DISP_RET_YEAR'], errors='coerce')
joined_df6['DISP_SCAN_MIS'] = pd.to_numeric(joined_df6['DISP_SCAN_MIS'], errors='coerce')
joined_df6['DISP_SCAN_MAN'] = pd.to_numeric(joined_df6['DISP_SCAN_MAN'], errors='coerce')

In [ ]:
joined_df6.info()

In [ ]:
joined_df6

In [ ]:
# Normalizing the counts and total costs 

list1 = joined_df6['REC_CREAT_COUNT']

def normalize_list(list1):
    list_norm_creat = minmax_scale(list1)
    return list_norm_creat

list_norm_creat_count = normalize_list(list1)

In [ ]:
joined_df6.columns

In [ ]:
list2 = joined_df6['REC_CREAT_COST']
list_norm_creat_cost = normalize_list(list2)

In [ ]:
list3 = joined_df6['REC_CLASS_COUNT']
list_norm_class_count = normalize_list(list3)

In [ ]:
list4 = joined_df6['REC_CLASS_COST']
list_norm_class_cost = normalize_list(list4)

In [ ]:
list5 = joined_df6['LOC_DOC_COUNT']
list_norm_locdoc_count = normalize_list(list5)

In [ ]:
list6 = joined_df6['LOC_DOC_COST']
list_norm_locdoc_cost = normalize_list(list6)

In [ ]:
list7 = joined_df6['DISP_DOC_COUNT']
list_norm_dispdoc_count = normalize_list(list7)

In [ ]:
list8 = joined_df6['DISP_DOC_COST']
list_norm_dispdoc_cost = normalize_list(list8)

In [ ]:
list9 = joined_df6['DISP_RET_COUNT']
list_norm_dispret_count = normalize_list(list9)

In [ ]:
list10 = joined_df6['DISP_RET_COST']
list_norm_dispret_cost = normalize_list(list10)

In [ ]:
list11 = joined_df6['DISP_SCAN_MIS_COUNT']
list_norm_dispscanmis_count = normalize_list(list11)

In [ ]:
list12 = joined_df6['DISP_SCAN_MIS_COST']
list_norm_dispscanmis_cost = normalize_list(list12)

In [ ]:
list13 = joined_df6['DISP_SCAN_MAN_COUNT']
list_norm_dispscanman_count = normalize_list(list13)

In [ ]:
list14 = joined_df6['DISP_SCAN_MAN_COST']
list_norm_dispscanman_cost = normalize_list(list14)

In [ ]:
joined_df6.head(3)

In [ ]:
joined_df6.insert(3, 'NORM_CREAT_COUNT', list_norm_creat_count)
joined_df6.head()

In [ ]:
joined_df6.insert(5, 'NORM_CREAT_COST', list_norm_creat_cost)
joined_df6.head()

In [ ]:
joined_df6.insert(8, 'NORM_CLASS_COUNT', list_norm_class_count)
joined_df6.head()

In [ ]:
joined_df6.insert(10, 'NORM_CLASS_COST', list_norm_class_cost)
joined_df6.head()

In [ ]:
joined_df6.insert(13, 'NORM_LOCDOC_COUNT', list_norm_locdoc_count)
joined_df6.head()

In [ ]:
joined_df6.insert(15, 'NORM_LOCDOC_COST', list_norm_locdoc_cost)
joined_df6.head()

In [ ]:
joined_df6.insert(18, 'NORM_DISPDOC_COUNT', list_norm_dispdoc_count)
joined_df6.head()

In [ ]:
joined_df6.insert(20, 'NORM_DISPDOC_COST', list_norm_dispdoc_cost)
joined_df6.head()

In [ ]:
joined_df6.insert(23, 'NORM_DISPRET_COUNT', list_norm_dispret_count)
joined_df6.head()

In [ ]:
joined_df6.insert(25, 'NORM_DISPRET_COST', list_norm_dispret_cost)
joined_df6.head()

In [ ]:
joined_df6.insert(28, 'NORM_DISPSCANMIS_COUNT', list_norm_dispscanmis_count)
joined_df6.head()

In [ ]:
joined_df6.insert(30, 'NORM_DISPSCANMIS_COST', list_norm_dispscanmis_cost)
joined_df6.head()

In [ ]:
joined_df6.insert(33, 'NORM_DISPSCANMAN_COUNT', list_norm_dispscanman_count)
joined_df6.head()

In [ ]:
joined_df6.insert(35, 'NORM_DISPSCANMAN_COST', list_norm_dispscanman_cost)
joined_df6.head()

In [ ]:
joined_df6.info()

In [ ]:
joined_df6

In [ ]:
# Given weights
creation_count_wt = 1
creation_cost_wt = 2
class_count_wt = 0.5
class_cost_wt = 1
loc_doc_count_wt = 1
loc_doc_cost_wt = 2
disp_doc_count_wt = 1
disp_doc_cost_wt = 2
disp_ret_count_wt =  0.5
disp_ret_cost_wt = 1
disp_scanmis_count_wt = 1 
disp_scanmis_cost_wt = 2
disp_scanman_count_wt = 1
disp_scanman_cost_wt = 2
more_1_module_wt = 1

In [ ]:
# Specifying transaction count weights (default = 1)
creat_ct_wt = 1
class_ct_wt = 1
locdoc_ct_wt = 1
dispdoc_ct_wt = 1
dispret_ct_wt = 1
dispscanmis_ct_wt = 1
dispscanman_ct_wt = 1

In [ ]:
joined_df6.isna().sum()

In [ ]:
# Creating a dataframe by replacing NaN with '0'
final_df = joined_df6.fillna(0)

In [ ]:
final_df.info()

In [ ]:
final_df.head(3)

In [ ]:
# Creating a column with 'WEIGHTED_SCORE'
final_df = final_df.assign(
    WEIGHTED_SCORE = final_df.REC_CREATION*final_df.NORM_CREAT_COUNT*creation_count_wt*creat_ct_wt + 
                     final_df.REC_CREATION*final_df.NORM_CREAT_COST*creation_cost_wt*creat_ct_wt + 
                     final_df.REC_CLASS*final_df.NORM_CLASS_COUNT*class_count_wt*class_ct_wt +
                     final_df.REC_CLASS*final_df.NORM_CLASS_COST*class_cost_wt*class_ct_wt +
                     final_df.LOC_DOC*final_df.NORM_LOCDOC_COUNT*loc_doc_count_wt*locdoc_ct_wt +  
                     final_df.LOC_DOC*final_df.NORM_LOCDOC_COST*loc_doc_cost_wt*locdoc_ct_wt + 
                     final_df.DISP_DOC*final_df.NORM_DISPDOC_COUNT*disp_doc_count_wt*dispdoc_ct_wt +
                     final_df.DISP_DOC*final_df.NORM_DISPDOC_COST*disp_doc_cost_wt*dispdoc_ct_wt +
                     final_df.DISP_RET_YEAR*final_df.NORM_DISPRET_COUNT*disp_ret_count_wt*dispret_ct_wt + 
                     final_df.DISP_RET_YEAR*final_df.NORM_DISPRET_COST*disp_ret_cost_wt*dispret_ct_wt + 
                     final_df.DISP_SCAN_MIS*final_df.NORM_DISPSCANMIS_COUNT*disp_scanmis_count_wt*dispscanmis_ct_wt + 
                     final_df.DISP_SCAN_MIS*final_df.NORM_DISPSCANMIS_COST*disp_scanmis_cost_wt*dispscanmis_ct_wt + 
                     final_df.DISP_SCAN_MAN*final_df.NORM_DISPSCANMAN_COUNT*disp_scanman_count_wt*dispscanman_ct_wt +
                     final_df.DISP_SCAN_MAN*final_df.NORM_DISPSCANMAN_COST*disp_scanman_cost_wt*dispscanman_ct_wt +
                     final_df.MORE_1_MODULE*more_1_module_wt
)

In [ ]:
final_df.info()

In [ ]:
final_df

In [ ]:
# User-wise list sorted by weighted score
user_list_df = final_df.sort_values(by = 'WEIGHTED_SCORE', ascending = False, ignore_index=True).round(4)
user_list_df

In [ ]:
# Prioritized User-wise list with weighted score
pri_user_list = user_list_df[['USER_ID','WEIGHTED_SCORE']]
pri_user_list[:20]

In [ ]:
# Exporting the user list as .csv file to bucket
file_name = 'user_list.csv'
user_list_df.to_csv(file_name)
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name, 'daen690-meraki-data', 'user_list.csv')

In [ ]:
# Exporting the styled image in local bucket
pri_user_list_styled = pri_user_list[:20].style.background_gradient()
pri_user_list_styled
dfi.export(pri_user_list_styled, 'top_users_list.png')

In [ ]:
# Export list as image

def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

fig,ax = render_mpl_table(pri_user_list[:20], header_columns=0, col_width=4.0)
fig.savefig("top_users_list.png")

img_data = io.BytesIO()
plt.savefig(img_data, format='png')
img_data.seek(0)

s3 = boto3.resource('s3')
bucket = s3.Bucket('daen690-meraki-dashboard-sample')
bucket.put_object(Body=img_data, ContentType='image/png', Key='top_users_list.png')

In [ ]:
p = sns.barplot(x='USER_ID', y='WEIGHTED_SCORE', data=pri_user_list[:50], color = 'blue')
p.set_xticklabels(p.get_xticklabels(),rotation=90)
p.set(xlabel='Users', ylabel='Weighted Score')
p.set_title('Top 50 Users with highest Weighted Score')
plt.gcf().set_size_inches(13, 5)

In [ ]:
p = sns.barplot(y='USER_ID', x='WEIGHTED_SCORE', data=pri_user_list[:20], color = 'blue')
p.set(ylabel='Users', xlabel='Weighted Score')
p.set_title('Top 20 Users with highest Weighted Score')
plt.gcf().set_size_inches(9, 10)

img_data = io.BytesIO()
plt.savefig(img_data, format='png')
img_data.seek(0)

s3 = boto3.resource('s3')
bucket = s3.Bucket('daen690-meraki-dashboard-sample')
bucket.put_object(Body=img_data, ContentType='image/png', Key='top_users_plot.png')

In [ ]:
pd.set_option('display.max_rows', 20)

In [ ]:
user_more_mod = user_list_df[user_list_df.MORE_1_MODULE==1].reset_index()
user_more_mod

In [ ]:
user_mod_list = user_more_mod[['USER_ID','WEIGHTED_SCORE']]
user_mod_list[:20]

In [ ]:
# Exporting the styled image in local bucket
user_mod_list_styled = user_mod_list[:20].style.background_gradient()
user_mod_list_styled
dfi.export(user_mod_list_styled, 'user_module_list.png')

In [ ]:
# Export list as image

def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

fig,ax = render_mpl_table(user_mod_list[:20], header_columns=0, col_width=4.0)
fig.savefig("user_module_list.png")

img_data = io.BytesIO()
plt.savefig(img_data, format='png')
img_data.seek(0)

s3 = boto3.resource('s3')
bucket = s3.Bucket('daen690-meraki-dashboard-sample')
bucket.put_object(Body=img_data, ContentType='image/png', Key='user_module_list.png')

In [ ]:
p = sns.barplot(x='USER_ID', y='WEIGHTED_SCORE', data=user_more_mod, color = 'brown')
p.set_xticklabels(p.get_xticklabels(),rotation=90)
p.set(xlabel='Users', ylabel='Weighted Score')
p.set_title('Users with errors in more than one module')
plt.gcf().set_size_inches(15, 5)

In [ ]:
p = sns.barplot(y='USER_ID', x='WEIGHTED_SCORE', data=user_more_mod[:20], color = 'brown')
p.set(xlabel='Weighted Score', ylabel= 'Users')
p.set_title('Top 20 Users with Errors in More than One Module')
plt.gcf().set_size_inches(9,10)

img_data = io.BytesIO()
plt.savefig(img_data, format='png')
img_data.seek(0)

s3 = boto3.resource('s3')
bucket = s3.Bucket('daen690-meraki-dashboard-sample')
bucket.put_object(Body=img_data, ContentType='image/png', Key='top_users_module_plot.png')

---

In [ ]:
# Creating a dataframe for location-wise users
loc_1 = loc_1.drop(columns='COUNT')
loc_2 = loc_2.drop(columns='COUNT')
loc_3 = loc_3.drop(columns='COUNT')
loc_4 = loc_4.drop(columns='COUNT')
loc_5 = loc_5.drop(columns='COUNT')
loc_6 = loc_6.drop(columns='COUNT')

In [ ]:
loc_1.columns = ['LOCATION', 'USER_ID']
loc_2.columns = ['LOCATION', 'USER_ID']
loc_3.columns = ['LOCATION', 'USER_ID']
loc_4.columns = ['LOCATION', 'USER_ID']
loc_5.columns = ['LOCATION', 'USER_ID']
loc_6.columns = ['LOCATION', 'USER_ID']

In [ ]:
loc_user_df = pd.concat([loc_1, loc_2, loc_3, loc_4, loc_5, loc_6]).drop_duplicates().sort_values(
                        by = ['LOCATION', 'USER_ID'], ascending = [True, True], ignore_index=True)
loc_user_df

In [ ]:
# Location-wise prioritized user list
loc_user_list = pd.merge(loc_user_df, final_df, how ='outer', on = 'USER_ID').sort_values(
    by = ['LOCATION', 'WEIGHTED_SCORE'], ascending = [True, False], ignore_index=True).round(4)
loc_user_list

In [ ]:
# Location-wise user list with weighted score
pri_locuser_list = loc_user_list[['LOCATION','USER_ID','WEIGHTED_SCORE']]
pri_locuser_list[:20]

In [ ]:
# Exporting the styled image in local bucket
pri_locuser_list_styled = pri_locuser_list[:20].style.background_gradient()
pri_locuser_list_styled
dfi.export(pri_locuser_list_styled, 'loc_user_list.png')

In [ ]:
# Export list as image

def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

fig,ax = render_mpl_table(pri_locuser_list[:20], header_columns=0, col_width=4.0)
fig.savefig("loc_user_list.png")

img_data = io.BytesIO()
plt.savefig(img_data, format='png')
img_data.seek(0)

s3 = boto3.resource('s3')
bucket = s3.Bucket('daen690-meraki-dashboard-sample')
bucket.put_object(Body=img_data, ContentType='image/png', Key='loc_user_list.png')

In [ ]:
# Exporting the location wise user list as .csv file to bucket
file_name = 'location_user_list.csv'
loc_user_list.to_csv(file_name)
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name, 'daen690-meraki-data', 'location_user_list.csv')

In [ ]:
# Location-wise list with total weighted score for location
loc_list = loc_user_list.groupby("LOCATION").agg(
            AGG_WT_SCORE = ("WEIGHTED_SCORE","sum"),
            USER_COUNT = ("WEIGHTED_SCORE","count")
).sort_values(by = 'AGG_WT_SCORE', ascending = False).round(2).reset_index()
loc_list['AVG_WT_SCORE'] = loc_list.AGG_WT_SCORE/loc_list.USER_COUNT
loc_list.index += 1
loc_list.AVG_WT_SCORE = loc_list.AVG_WT_SCORE.round(2)
loc_list

In [ ]:
# Exporting the location wise list as .csv file to bucket
file_name = 'location_list.csv'
loc_list.to_csv(file_name)
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name, 'daen690-meraki-data', 'location_list.csv')

In [ ]:
# Top 20 Locations with highest Aggregate Weighted Score
loc_list[:20]

In [ ]:
# Exporting the styled image in local bucket
loc_list_styled = loc_list[:20].style.background_gradient()
loc_list_styled
dfi.export(loc_list_styled, 'loc_list.png')

In [ ]:
# Export list as image

def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

fig,ax = render_mpl_table(loc_list[:20], header_columns=0, col_width=4.0)
fig.savefig("loc_list.png")

img_data = io.BytesIO()
plt.savefig(img_data, format='png')
img_data.seek(0)

s3 = boto3.resource('s3')
bucket = s3.Bucket('daen690-meraki-dashboard-sample')
bucket.put_object(Body=img_data, ContentType='image/png', Key='loc_list.png')

In [ ]:
p = sns.barplot(x='LOCATION', y='AGG_WT_SCORE', data=loc_list[:50], color = 'navy')
p.set_xticklabels(p.get_xticklabels(),rotation=90)
p.set(xlabel='Location', ylabel='Aggregate Weighted Score')
p.set_title('Top 50 Locations with Highest Aggregate Weighted Score')
plt.gcf().set_size_inches(14, 6)

In [ ]:
p = sns.barplot(y='LOCATION', x='AGG_WT_SCORE', data=loc_list[:20:], color = 'navy')
p.set(ylabel='Location', xlabel='Aggregate Weighted Score')
p.set_title('Top 20 Locations with Highest Aggregate Weighted Score')
plt.gcf().set_size_inches(11,11)

img_data = io.BytesIO()
plt.savefig(img_data, format='png')
img_data.seek(0)

s3 = boto3.resource('s3')
bucket = s3.Bucket('daen690-meraki-dashboard-sample')
bucket.put_object(Body=img_data, ContentType='image/png', Key='top_locations_plot.png')

In [ ]:
eruser_ct_rec = sum(list_module_rec)
eruser_ct_rec 

In [ ]:
eruser_ct_loc = sum(list_module_loc)
eruser_ct_loc

In [ ]:
eruser_ct_disp = sum(list_module_disp)
eruser_ct_disp

In [ ]:
eruser_ct_tot = user_list_df.USER_ID.nunique()
eruser_ct_tot

In [ ]:
user_ct_rec = df_receiving.OPR_ID.nunique()
user_ct_rec

In [ ]:
user_ct_loc = df_locations.ENTERED_BY.nunique()
user_ct_loc

In [ ]:
user_ct_disp = df_disposals.USER_ID.nunique()
user_ct_disp

In [ ]:
d1 = {'MODULE':['Receiving', 'Transfering', 'Disposal'],
     'TOTAL_USERS':[user_ct_rec, user_ct_loc, user_ct_disp],
     'USERS_W_ERROR':[eruser_ct_rec , eruser_ct_loc, eruser_ct_disp]}
df_summary1 = pd.DataFrame(data=d1)
df_summary1

In [ ]:
# Export list as image

def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

fig,ax = render_mpl_table(df_summary1, header_columns=0, col_width=4.0)
fig.savefig("total_error_user_list.png")

img_data = io.BytesIO()
plt.savefig(img_data, format='png')
img_data.seek(0)

s3 = boto3.resource('s3')
bucket = s3.Bucket('daen690-meraki-dashboard-sample')
bucket.put_object(Body=img_data, ContentType='image/png', Key='total_error_user_list.png')

In [ ]:
d2 = {'MODULE':['Receiving', 'Transfering', 'Disposal', 'Receiving', 'Transfering', 'Disposal'],
     'USERS':['TOTAL', 'TOTAL', 'TOTAL', 'ERROR', 'ERROR', 'ERROR'],
     'COUNT':[user_ct_rec, user_ct_loc, user_ct_disp, eruser_ct_rec , eruser_ct_loc, eruser_ct_disp],}
df_summary2 = pd.DataFrame(data=d2)
df_summary2

In [ ]:
d3 = {'SUMMARY_STATISTIC':['Total Users with Errors', 'Users with Errors >1 Module', 
                           'Locations with Errors'],
     'COUNT':[user_list_df.shape[0], user_more_mod.shape[0], loc_list.shape[0]]}
df_summary3 = pd.DataFrame(data=d3)
df_summary3

In [ ]:
# Export list as image

def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

fig,ax = render_mpl_table(df_summary3, header_columns=0, col_width=4.0)
fig.savefig("summary_table.png")

img_data = io.BytesIO()
plt.savefig(img_data, format='png')
img_data.seek(0)

s3 = boto3.resource('s3')
bucket = s3.Bucket(dashboard_bucket)
bucket.put_object(Body=img_data, ContentType='image/png', Key='summary_table.png')

In [ ]:
p = sns.barplot(x='MODULE', y='COUNT', hue = 'USERS', data=df_summary2)
p.set_title('Module-wise Total Users and Users with Errors Count')
plt.gcf().set_size_inches(7, 7)

img_data = io.BytesIO()
plt.savefig(img_data, format='png')
img_data.seek(0)

s3 = boto3.resource('s3')
bucket = s3.Bucket(dashboard_bucket)
bucket.put_object(Body=img_data, ContentType='image/png', Key='total_error_user_plot.png')